# Lab-04-1 Multivariate Linear Regression
- Simple Linear Regression 복습
- Multivariate Linear Regression 이론
- Naive Data Representation
- Matrix Data Representation
- Multivariate Linear Regression
- `nn.Module` 소개
- `F.mse_loss` 소개

In [1]:
import torch
from torch import optim

## Hypothesis Function

$H(x) = w_1x_1 + w_2x_2 + w_3x_3 + b$ -> $H(x) = Wx + b$

In [2]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b     # or .mm or @
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print("Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}".format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()  # detach는 grad 소거
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712891
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371010
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost: 10.445305
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391228
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493135
Epoch   10/20 hypo

## `nn.Module`

- `nn.Module`을 상속해서 모델 생성
- `nn.Linear(3, 1)`
    - 입력 차원: 3
    - 출력 차원: 1
- Hypothesis 계산은 `forward()`에서!
- Gradient 계산은 PyTorch가 알아서 해준다 `backward()`

In [3]:
import torch.nn as nn

class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
    
    def forward(self, x):
        return self.linear(x)

#hypothesis = model(x_train)

## `F.mse_loss`

- `torch.nn.functional`에서 제공하는 loss function 사용
- 쉽게 다른 loss와 교체 가능! (`l1_loss`, `smooth_l1_loss` 등...)

In [4]:
import torch.nn.functional as F

In [11]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
#W = torch.zeros((3, 1), requires_grad=True)
#b = torch.zeros(1, requires_grad=True)

# optimizer 설정
model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    #hypothesis = x_train.matmul(W) + b
    hypothesis = model(x_train)
    
    # cost 계산
    #cost = torch.mean((hypothesis - y_train) ** 2)
    cost = F.mse_loss(hypothesis, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print("Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}".format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()  # detach는 grad 소거
    ))

Epoch    0/20 hypothesis: tensor([-43.8276, -49.3782, -50.3708, -54.4904, -37.1691]) Cost: 48239.851562
Epoch    1/20 hypothesis: tensor([41.9473, 53.7167, 51.2104, 56.1289, 41.4664]) Cost: 15120.916016
Epoch    2/20 hypothesis: tensor([ 89.9695, 111.4358, 108.0820, 118.0606,  85.4915]) Cost: 4739.886230
Epoch    3/20 hypothesis: tensor([116.8553, 143.7506, 139.9223, 152.7339, 110.1395]) Cost: 1485.985718
Epoch    4/20 hypothesis: tensor([131.9078, 161.8424, 157.7486, 172.1462, 123.9390]) Cost: 466.059662
Epoch    5/20 hypothesis: tensor([140.3351, 171.9714, 167.7288, 183.0144, 131.6649]) Cost: 146.367218
Epoch    6/20 hypothesis: tensor([145.0532, 177.6422, 173.3164, 189.0991, 135.9903]) Cost: 46.160538
Epoch    7/20 hypothesis: tensor([147.6947, 180.8171, 176.4447, 192.5057, 138.4120]) Cost: 14.751104
Epoch    8/20 hypothesis: tensor([149.1736, 182.5946, 178.1961, 194.4129, 139.7678]) Cost: 4.905950
Epoch    9/20 hypothesis: tensor([150.0016, 183.5897, 179.1767, 195.4807, 140.5268]) 

## What's Next?
- 지금까지 적은 양의 데이터를 가지고 학습했습니다.
- 하지만 딥러닝은 많은 양의 데이터와 함께할 때 빛을 발합니다. (컴퓨터에 다 넣지 못할 정도의)
- PyTorch에서는 많은 양의 데이터를 어떻게 다룰까요?